In [79]:
%pip install transformers
%pip install torch
%pip install sentencepiece
%pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` befor

## Модели

### Модель: rubert-tiny2-sentence-compression

In [70]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

rubert_tiny2_name = 'cointegrated/rubert-tiny2-sentence-compression'
rubert_tiny2_model = AutoModelForTokenClassification.from_pretrained(rubert_tiny2_name)
rubert_tiny2_tokenizer = AutoTokenizer.from_pretrained(rubert_tiny2_name)

def rubert_tiny2_compress(text, threshold=0.5, keep_ratio=None, all=False):
    """ Compress a sentence by removing the least important words.
    Parameters:
        threshold: cutoff for predicted probabilities of word removal
        keep_ratio: proportion of words to preserve
    By default, threshold of 0.5 is used.
    """
    with torch.inference_mode():
        tok = rubert_tiny2_tokenizer(text, return_tensors='pt').to(rubert_tiny2_model.device)
        proba = torch.softmax(rubert_tiny2_model(**tok).logits, -1).cpu().numpy()[0, :, 1]
    if keep_ratio is not None:
        threshold = sorted(proba)[int(len(proba) * keep_ratio)]
    kept_toks = []
    keep = False
    prev_word_id = None
    
    if (all):
        mp = {}
        for word_id, score, token in zip(tok.word_ids(), proba, tok.input_ids[0]):
            mp[score] = token
        
        list_ = list()
        for i in sorted(mp):
            list_.append(mp[i])
        
        return rubert_tiny2_tokenizer.decode(list_, skip_special_tokens=True)
                
    for word_id, score, token in zip(tok.word_ids(), proba, tok.input_ids[0]):
        if word_id is None:
            keep = True
        elif word_id != prev_word_id:
            keep = score < threshold
        if keep:
            kept_toks.append(token)
        prev_word_id = word_id
    return rubert_tiny2_tokenizer.decode(kept_toks, skip_special_tokens=True)

### Модель: rut5-base-absum

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

rut5_base_name = 'cointegrated/rut5-base-absum'
rut5_base_model = T5ForConditionalGeneration.from_pretrained(rut5_base_name)
rut5_base_tokenizer = T5Tokenizer.from_pretrained(rut5_base_name)

def rut5_base_compress(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = rut5_base_tokenizer(text, return_tensors='pt', padding=True).to(rut5_base_model.device)
    with torch.inference_mode():
        out = rut5_base_model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return rut5_base_tokenizer.decode(out[0], skip_special_tokens=True)

### Модель: mT5_multilingual_XLSum (-)

In [ ]:
# import re
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

# mT5_multilingual_name = "csebuetnlp/mT5_multilingual_XLSum"
# mT5_multilingual_tokenizer = AutoTokenizer.from_pretrained(mT5_multilingual_name)
# mT5_multilingual_model = AutoModelForSeq2SeqLM.from_pretrained(mT5_multilingual_name)

# def mT5_multilingual_compress(
#     text
# ):

#   input_ids = mT5_multilingual_tokenizer(
#       [WHITESPACE_HANDLER(text)],
#       return_tensors="pt",
#       padding="max_length",
#       truncation=True,
#       max_length=512
#   )["input_ids"]

#   output_ids = mT5_multilingual_model.generate(
#       input_ids=input_ids,
#       max_length=84,
#       no_repeat_ngram_size=2,
#       num_beams=4
#   )[0]

#   summary = mT5_multilingual_tokenizer.decode(
#       output_ids,
#       skip_special_tokens=True,
#       clean_up_tokenization_spaces=False
#   )

#   return summary

### Модель: rubert_telegram_headlines

In [ ]:
# from transformers import AutoTokenizer, EncoderDecoderModel

# rubert_telegram_name = "IlyaGusev/rubert_telegram_headlines"
# rubert_telegram_tokenizer = AutoTokenizer.from_pretrained(rubert_telegram_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
# rubert_telegram_model = EncoderDecoderModel.from_pretrained(rubert_telegram_name)

# def rubert_telegram_compress(
#     text
# ):
#   input_ids = rubert_telegram_tokenizer(
#       [text],
#       add_special_tokens=True,
#       max_length=256,
#       padding="max_length",
#       truncation=True,
#       return_tensors="pt",
#   )["input_ids"]

#   output_ids = rubert_telegram_model.generate(
#       input_ids=input_ids,
#       max_length=64,
#       no_repeat_ngram_size=3,
#       num_beams=10,
#       top_p=0.95
#   )[0]

#   return rubert_telegram_tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
  

In [58]:
def ref(comment):
  return comment.lower().replace(',', '.').replace('!', '.').replace('?', '.').split('.')

In [15]:
def print_compress(comments): 
  for comment in comments:
    print(comment, '\n')
    rubert_tiny2_compress_comment = str(rubert_tiny2_compress(comment, keep_ratio= min(3/len(comment.split()), 0.9999)))
    rut5_base_compress_comment = str(rut5_base_compress(comment, compression=min(3/len(comment.split()), 0.9999)))

    print("rubert_tiny2_compress: ", rubert_tiny2_compress(comment)," ## ", rubert_tiny2_compress_comment)
    print("rut5_base_compress: ", rut5_base_compress(comment)," ## ", rut5_base_compress_comment)

    print('\n', '='*40, '\n')

In [65]:
def test_rubert_tiny2_compress(comments):
    for comment in comments:
        print(comment)
        comment_len = len(comment.split())
        for i in range(comment_len):
            print(i + 1, ":", rubert_tiny2_compress(comment, keep_ratio=(i+0.9999)/comment_len))
        print('\n', '='*40)

In [76]:
def test_rut5_base_compress(comments):
    for comment in comments:
        print(comment)
        comment_len = len(comment.split())
        for i in range(comment_len):
            print(i + 1, ":", rut5_base_compress(comment, compression=(i+0.9999)/comment_len))
        print('\n', '='*40)

In [92]:
def tiny_test_DF_rubert_tiny2_compress(comments):
    comment_objects_small = [] 
    comment_objects_medium = []
    comment_objects_auto = []
    
    for comment in comments:
        comment_len = len(comment.split())
        comment_objects_small.append(rubert_tiny2_compress(comment, keep_ratio=min(2/comment_len,0.999)))
        comment_objects_medium.append(rubert_tiny2_compress(comment, keep_ratio=1/2))
        comment_objects_auto.append(rubert_tiny2_compress(comment))
    
    return comment_objects_small, comment_objects_medium, comment_objects_auto

In [125]:
import pandas as pd
import numpy as np

In [94]:
comments = pd.read_csv("wildberries_with_predict.csv")

comment_objects_small, comment_objects_medium, comment_objects_auto = tiny_test_DF_rubert_tiny2_compress(comments['text'].to_list())


In [128]:
comment_objects_DF = pd.DataFrame(
    data=np.array([comments['text'].to_list(), comment_objects_small, comment_objects_medium,comment_objects_auto]).T,
    columns=['text', 'small', 'medium', 'auto']
)


In [129]:
comment_objects_DF

,text,small,medium,auto
0,"Всё норм. Мужу понравилось, как всегда.",..,Всё норм..,.
1,Все пришло попробовал отлично,попробовал отлично,попробовал отлично,Все пришло попробовал отлично
2,"Супер! Оригинал, выгодней намного чем в магази...",Супер!!,Супер! Оригинал.!,Супер! Оригинал!
3,"Шикарные лезвия, 4 штуки в упаковке (фоткала у...",лезвия.,"Шикарные лезвия, 4 штуки в упаковке. Качество ...","Шикарные лезвия, 4 штуки в упаковке. Качество ..."
4,На первый взгляд очень даже хорошие кассеты. П...,На..,На первый кассеты...,На..
...,...,...,...,...
70078,Для тренировок в сммый раз,Для тренировок раз,Для тренировок раз,Для тренировок в сммый раз
70079,Хорошая майка,Хорошая майка,майка,Хорошая майка
70080,Майка не понравилась-очень тонкий материал!,Майка!,Майка не!,Майка не понравилась - тонкий материал!
70081,На рост 190 см Вес 94 кг .подошел отлично разм...,На,На рост 190 см Вес 94 кг. ХL,На рост 190 см Вес кг


In [130]:
comment_objects_DF.to_csv('wildberries_comment_objects_example_test.csv')